# WANN experiments on UCI dataset

In [11]:
import copy
import sys

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.constraints import MinMaxNorm
from tensorflow.keras.optimizers import Adam

sys.path.append("../wann")
from utils import superconduct, domain, BaggingModels, cross_val
from uci_experiments import run_uci_experiments
from methods import *

from warnings import filterwarnings
filterwarnings('ignore')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Load Dataset

In [12]:
source = 0  # possible values: 0, 1, 2, 3
target = 2  # possible values: 0, 1, 2, 3

N = 10   # Number of labeled target data

data, X, y, cuts, split_col = superconduct()
shape = X.shape[1]

src_index = domain(data, cuts, split_col, source)
tgt_index = domain(data, cuts, split_col, target)

np.random.seed(0)
tgt_train_index, tgt_test_index = train_test_split(tgt_index, train_size=N)
train_index = np.concatenate((src_index, tgt_train_index))

std_sc = StandardScaler()
std_sc.fit(X[train_index])
X = std_sc.transform(X)
y = (y - y[train_index].mean()) / y[train_index].std()

## Base Estimator

In [13]:
def get_base_model(shape, activation=None, C=1, name="BaseModel"):
    inputs = Input(shape=(shape,))
    modeled = Dense(100, activation='relu',
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(inputs)
    modeled = Dense(1, activation=activation,
                    kernel_constraint=MinMaxNorm(0, C),
                    bias_constraint=MinMaxNorm(0, C))(modeled)
    model = Model(inputs, modeled, name=name)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def get_encoder(shape, C=1, name="encoder"):
    inputs = Input(shape=(shape,))
    modeled = Dense(100, activation='relu',
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(inputs)
    model = Model(inputs, modeled)
    model.compile(optimizer="adam", loss='mean_squared_error')
    return model

def get_task(shape, C=1, activation=None, name="task"):
    inputs = Input(shape=(shape,))
    modeled = Dense(1, activation=activation,
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(inputs)
    model = Model(inputs, modeled)
    model.compile(optimizer="adam", loss='mean_squared_error')
    return model


base_estimator = BaggingModels(func=get_base_model,
                               n_models=1,
                               n_jobs=None,
                               shape=shape,
                               C=1,
                               random_state=0)
fit_params = dict(epochs=200,
                  batch_size=1000,
                  verbose=0)

## Examples

### Target only

In [4]:
tgt_only = copy.deepcopy(base_estimator)
tgt_only.fit(X[tgt_train_index], y[tgt_train_index], **fit_params)

y_pred = tgt_only.predict(X)
score= mean_squared_error(y[tgt_test_index], y_pred[tgt_test_index])
print('Target score: %.3f'%score)

Target score: 0.372


### Source only

In [5]:
src_only = copy.deepcopy(base_estimator)
src_only.fit(X[src_index], y[src_index], **fit_params)

y_pred = src_only.predict(X)
score= mean_squared_error(y[tgt_test_index], y_pred[tgt_test_index])
print('Target score: %.3f'%score)

Target score: 0.501


### No reweight

In [6]:
no_reweight = copy.deepcopy(base_estimator)
no_reweight.fit(X[train_index], y[train_index], **fit_params)

y_pred = no_reweight.predict(X)
score= mean_squared_error(y[tgt_test_index], y_pred[tgt_test_index])
print('Target score: %.3f'%score)

Target score: 0.453


### BalancedWeighting

In [8]:
np.random.seed(0)
tf.random.set_seed(0)

balanced = BalancedWeighting(get_base_model)
balanced.fit(X, y, [src_index, tgt_train_index], **fit_params)

y_pred = balanced.predict(X)
score= mean_squared_error(y[tgt_test_index], y_pred[tgt_test_index])
print('Target score: %.3f'%score)

Target score: 0.390


### DANN

In [9]:
dann = BaggingModels(DANN, n_models=1, n_jobs=None, random_state=0,
                     get_encoder=get_encoder, get_task=get_task, lambda_=0.05)

dann.fit(X, y, index=[src_index, tgt_index, tgt_train_index], **fit_params)

y_pred = dann.predict(X)
score = mean_squared_error(y[tgt_test_index], y_pred[tgt_test_index])
print('Target score: %.3f'%score)

Target score: 0.745


### ADDA

In [15]:
adda = BaggingModels(ADDA, n_models=1, n_jobs=None, random_state=0, optimizer=Adam(0.00001),
                     get_encoder=get_encoder, get_task=get_task, get_discriminer=get_task)

adda.fit(X, y, index=[src_index, tgt_index, tgt_train_index], **fit_params)

y_pred = adda.predict(X)
score = mean_squared_error(y[tgt_test_index], y_pred[tgt_test_index])
print('Target score: %.3f'%score)

Target score: 0.793


### MCD

In [16]:
mcd = BaggingModels(MCD, n_models=1, n_jobs=None, random_state=0,
                     get_encoder=get_encoder, get_task=get_task, lambda_=0.1)

mcd.fit(X, y, index=[src_index, tgt_index, tgt_train_index], **fit_params)

y_pred = mcd.predict(X)
score = mean_squared_error(y[tgt_test_index], y_pred[tgt_test_index])
print('Target score: %.3f'%score)

Target score: 0.395


### MDD

In [18]:
mdd = BaggingModels(MDD, n_models=1, n_jobs=None, random_state=0,
                     get_encoder=get_encoder, get_task=get_task, lambda_=0.1)

mdd.fit(X, y, index=[src_index, tgt_index, tgt_train_index], **fit_params)

y_pred = mdd.predict(X)
score = mean_squared_error(y[tgt_test_index], y_pred[tgt_test_index])
print('Target score: %.3f'%score)

Target score: 0.820


### WANN

In [19]:
wann = BaggingModels(WANN, n_models=1, n_jobs=None, random_state=0,
                     get_base_model=get_base_model, C=1, C_w=0.1)

wann.fit(X, y, index=[src_index, tgt_train_index], **fit_params)

y_pred = wann.predict(X)
score = mean_squared_error(y[tgt_test_index],
                           y_pred[tgt_test_index])
print('Target score: %.3f'%score)

Target score: 0.289


## Run Experiments

In [22]:
# Add unlabeled sample 

df = run_uci_experiments(method="WANN",
                         get_base_model=get_base_model,
                         get_encoder=get_encoder,
                         get_task=get_task,
                         C=1,
                         C_w=0.1,
                         lambda_=0.1,
                         sigma=0.1,
                         epochs=200,
                         batch_size=1000,
                         n_models=1,
                         n_jobs=None,
                         n_target_labeled=10,
                         random_state=0,
                         save=False)

Experiment for method: WANN


############# 0 #############
--------- 1 ----------
Target_score: 0.233
--------- 2 ----------
Target_score: 0.289
--------- 3 ----------
Target_score: 0.334
############# 1 #############
--------- 0 ----------
Target_score: 0.435
--------- 2 ----------
Target_score: 0.258
--------- 3 ----------
Target_score: 0.371
############# 2 #############
--------- 0 ----------
Target_score: 0.599
--------- 1 ----------
Target_score: 0.395
--------- 3 ----------
Target_score: 0.355
############# 3 #############
--------- 0 ----------
Target_score: 0.636
--------- 1 ----------
Target_score: 0.502
--------- 2 ----------
Target_score: 0.325


### Launch all experiments (all methods, 10 times)
Uncomment cell below to launch experiments

In [12]:
# %run -i ..\wann\uci_experiments